In [1]:
%load_ext blackcellmagic

import requests
import pandas as pd
import re
from time import sleep


In [2]:
url = "https://elephrame.com/gather"
payload = {"page": 5, "contentType": "blm", "search": ""}

header = {
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Referer": "https://elephrame.com/textbook/BLM",
    "Accept": "text/html, */*; q=0.01",
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12) "
        "AppleWebKit/602.1.50 (KHTML, like Gecko) "
        "Version/10.0 Safari/602.1.50"
    ),
    "Origin": "https://elephrame.com",
}

In [3]:
def scrape_elephrame(html):
    events = []
    for section in html.split('item-chart-header"')[1:]:
        id_no =    "".join(re.findall('id="(.*?)"', section))
        location = "".join(re.findall('item-protest-location">(.*?)<', section))
        size = "".join(re.findall("Participant\(s\):</b> (.*?)<", section))
        date = "".join(re.findall('protest-start">.*?, (.*?)<', section))
        time = "".join(re.findall("<b>Time:<\/b>(.*?)<\/li", section, re.DOTALL))
        description = "".join(
            re.findall("<b>Description:<\/b>(.*?)<\/li", section, re.DOTALL)
        )
        subject = "".join(re.findall("Subject.*?>(.*?)<", section))
        links = re.findall('href="(.*?)"', section)
        events.append((id_no, location, size, date, subject, description, time, links))
    new_df = pd.DataFrame(
        events,
        columns=["ID", "Location", "Size", "Date", "Subject", "Description", "Time", "Links"],
    )
    return new_df

In [4]:
dfs = []

for page in range(0, 40):
    payload["page"] = page
    r = requests.post(url, data=payload, headers=header)
    html = r.text
    new_df = scrape_elephrame(html)
    dfs.append(new_df)
    sleep(3)

In [5]:
ele_df = pd.concat(dfs)

In [6]:
ele_df['Collection'] = 'Elephrame'

In [7]:
ele_df.sample(3)

,ID,Location,Size,Date,Subject,Description,Time,Links,Collection
9,3427,"Elmira, NY",40-50 (est.) demonstrators,"June 5, 2020","#WhiteCoatsForBlackLives, George Floyd, Breon...",Walkout of healthcare workers at Arnot Ogden ...,Unclear,[https://www.weny.com/story/42216709/health-ca...,Elephrame
0,3254,"Williamsburg, VA",Hundreds of demonstrators,"June 1, 2020","George Floyd, Breonna Taylor, Ahmaud Arbery, ...",No details included for this demonstration.,Evening,[https://wydaily.com/local-news/2020/06/01/i-c...,Elephrame
1,3363,"Carmel, NY",100 (est.) demonstrators,"June 3, 2020","George Floyd, Breonna Taylor, Ahmaud Arbery, ...",No details included for this demonstration.,Afternoon,[https://www.lohud.com/story/news/local/putnam...,Elephrame


In [8]:
ele_df.to_csv('Elephrame.csv', index=False)